In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Lambda
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.layers import MaxPooling2D
from sklearn.model_selection import train_test_split
import tensorflow as tf
import numpy as np
from tensorflow import keras
import tensorflow.keras.backend as K
import matplotlib.pyplot as plt
from numpy import load
import os
import math

Build the model

In [ ]:
def build_siamese_model(inputShape, embeddingDim=48):
    # Adapted from: https://www.pyimagesearch.com/2020/11/30/siamese-networks-with-keras-tensorflow-and-deep-learning/

    # specify the inputs for the feature extractor network
    inputs = Input(inputShape)
    # define the first set of CONV => RELU => POOL => DROPOUT layers
    x = Conv2D(64, (3, 4), padding="same", activation="relu")(inputs)
    x = MaxPooling2D(pool_size=(3, 4))(x)
    x = Dropout(0.3)(x)
    # second set of CONV => RELU => POOL => DROPOUT layers
    x = Conv2D(64, (3, 4), padding="same", activation="relu")(x)
    x = MaxPooling2D(pool_size=3)(x)
    x = Dropout(0.3)(x)

    x = Conv2D(64, (2, 2), padding="same", activation="relu")(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Dropout(0.3)(x)
    # second set of CONV => RELU => POOL => DROPOUT layers
    x = Conv2D(64, (2, 2), padding="same", activation="relu")(x)
    x = MaxPooling2D(pool_size=2)(x)
    x = Dropout(0.3)(x)
    # prepare the final outputs
    pooledOutput = GlobalAveragePooling2D()(x)


    outputs = Dense(embeddingDim)(pooledOutput)
    # build the model
    model = Model(inputs, outputs)
    # model.summary()
    # return the model to the calling function
    return model
  
def euclidean_distance(vectors):
    # unpack the vectors into separate lists
    (featsA, featsB) = vectors
    # compute the sum of squared distances between the vectors
    sumSquared = K.sum(K.square(featsA - featsB), axis=1, keepdims=True)
    # return the euclidean distance between the vectors
    return K.sqrt(K.maximum(sumSquared, K.epsilon()))

img_shape = (217, 334, 1)

Load the batched dataset

In [ ]:
batch_x_train = np.load("/content/gdrive/MyDrive/Music Recommendation/datasets/32k/batch_x_train.npz")
batch_y_train = np.load("/content/gdrive/MyDrive/Music Recommendation/datasets/32k/batch_y_train.npz")
batch_x_test  = np.load("/content/gdrive/MyDrive/Music Recommendation/datasets/32k/batch_x_test.npz")
batch_y_test  = np.load("/content/gdrive/MyDrive/Music Recommendation/datasets/32k/batch_y_test.npz")

Define the Pairs Generator Class to load the batched dataset pairs

In [ ]:
class PairsGenerator(keras.utils.Sequence):
    def __init__ (self, x_dataset, y_dataset):
        self.x_dataset = x_dataset
        self.y_dataset = y_dataset
    
    def __len__(self):
        return len(self.x_dataset)
    
    def __getitem__ (self, idx):
        return [self.x_dataset['arr_'+str(idx)][:,0], self.x_dataset['arr_'+str(idx)][:,1]], self.y_dataset['arr_'+str(idx)]
        

Instansiate the batch generators

In [ ]:
training_batch_generator = PairsGenerator(batch_x_train, batch_y_train)
testing_batch_generator = PairsGenerator(batch_x_test, batch_y_test)

Compile the model

In [ ]:
imgA = Input(shape=img_shape)
imgB = Input(shape=img_shape)


featureExtractor = build_siamese_model(img_shape)
featsA = featureExtractor(imgA)
featsB = featureExtractor(imgB)
distance = Lambda(euclidean_distance)([featsA, featsB])
outputs = Dense(1, activation="sigmoid")(distance)

model = Model(inputs=[imgA, imgB], outputs=outputs)

epochs = 180  
learning_rate = 0.0008

model.compile(loss="binary_crossentropy", optimizer=keras.optimizers.Adam(
    learning_rate=learning_rate),
    metrics=["accuracy"])

Start training

In [ ]:
checkpoint_filepath = '/checkpoint_directory/'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=False,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)
model.fit(
    training_batch_generator,
    validation_data=testing_batch_generator,
    batch_size=32, 
    epochs=epochs,
    callbacks=[model_checkpoint_callback])